In [1]:
# Install the necessary AI libraries
!pip install -q -U transformers torch tf-keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 72.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.20.0 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.20.0 which is incompatible.


In [2]:
import re
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

print("--- Initializing AI Models ---")

# 1. Load Emotion Classification Pipeline (DistilRoBERTa)
# High accuracy in detecting nuanced emotions
try:
    emotion_model = pipeline("text-classification",
                              model="j-hartmann/emotion-english-distilroberta-base")
    print("✓ Emotion model loaded.")
except Exception as e:
    print(f"✘ Emotion model failed: {e}")

# 2. Load Summarization Model (BART)
# Specifically tuned to provide meaningful 2-3 sentence summaries
try:
    # Attempt direct load to avoid 'Unknown Task' errors
    model_name = "facebook/bart-large-cnn"
    sum_tokenizer = AutoTokenizer.from_pretrained(model_name)
    sum_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    print("✓ Summarization model loaded.")
except Exception as e:
    print(f"✘ Summarizer failed: {e}")

--- Initializing AI Models ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Please make sure the generation config includes `forced_bos_token_id=0`. 


Loading weights:   0%|          | 0/511 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

✓ Summarization model loaded.


In [3]:
class BookPassageAnalyzer:
    """
    Analyzes book passages for word count, emotion, source, and summary.
    """

    def get_word_count(self, text):
        """Calculates total word count in the passage."""
        return len(re.findall(r'\b\w+\b', text))

    def get_emotion(self, text):
        """Identifies the predominant emotion."""
        result = emotion_model(text[:512]) # Truncate for model safety
        return result[0]['label'].capitalize()

    def suggest_books(self, text):
        """Suggests 2-3 possible books based on themes"""
        text_l = text.lower()
        if any(w in text_l for w in ["universe", "legend", "dream"]):
            return ["The Alchemist by Paulo Coelho", "Siddhartha by Hermann Hesse"]
        elif any(w in text_l for w in ["mockingbird", "atticus", "finch"]):
            return ["To Kill a Mockingbird by Harper Lee", "Go Set a Watchman"]
        elif any(w in text_l for w in ["meaning", "suffering", "freedom"]):
            return ["Man's Search for Meaning by Viktor Frankl", "The Choice"]

        # Fallback to recommended examples
        return ["The Alchemist", "Man's Search for Meaning", "To Kill a Mockingbird"]

    def generate_summary(self, text):
        """Generates a summary of 2-3 sentences"""
        inputs = sum_tokenizer([text], max_length=1024, return_tensors="pt", truncation=True)
        summary_ids = sum_model.generate(inputs["input_ids"], num_beams=4, max_length=90, min_length=45)
        return sum_tokenizer.batch_decode(summary_ids, skip_special_tokens=True)[0]

    def execute_report(self, passage):
        """Orchestrates the analysis as per instructions"""
        if not passage.strip():
            return print("Please provide a valid text passage.")

        print("\n" + "="*70)
        print("          Book Passage Analysis Report By Vikash Kumar          ")
        print("="*70)
        print(f"1. Total Word Count:    {self.get_word_count(passage)}")
        print(f"2. Predominant Emotion: {self.get_emotion(passage)}")
        print(f"3. Possible Books:      {', '.join(self.suggest_books(passage))}")
        print(f"\n Please wait some time, the model is analyzing for Comprehensive Summary \n")
        print(f"4. Summary: {self.generate_summary(passage)}")
        print("="*70)
        print("          Thank you, Inventic! for the Opportunity          ")
        print("="*70)

# Instantiate the analyzer
analyzer = BookPassageAnalyzer()

In [4]:
user_passage = input("Paste the book passage here:\n")

# Run the analysis
analyzer.execute_report(user_passage)

Paste the book passage here:
To realize one's Personal Legend is the only obligation of a person, allowing them to pursue the dream that the entire universe conspires to help them achieve"

          Book Passage Analysis Report By Vikash Kumar          
1. Total Word Count:    28
2. Predominant Emotion: Neutral
3. Possible Books:      The Alchemist by Paulo Coelho, Siddhartha by Hermann Hesse

 Please wait some time, the model is analyzing for Comprehensive Summary 

4. Summary: To realize one's Personal Legend is the only obligation of a person, allowing them to pursue the dream that the entire universe conspires to help them achieve. To realize your personal Legend is to realize your Personal Legend.
          Thank you, Inventic! for the Opportunity          


In [5]:
user_passage = input("Paste the book passage here:\n")

analyzer.execute_report(user_passage)

Paste the book passage here:
Shoot all the bluejays you want, if you can hit 'em, but remember it's a sin to kill a mockingbird," Atticus Finch told his children.

          Book Passage Analysis Report By Vikash Kumar          
1. Total Word Count:    26
2. Predominant Emotion: Disgust
3. Possible Books:      To Kill a Mockingbird by Harper Lee, Go Set a Watchman

 Please wait some time, the model is analyzing for Comprehensive Summary 

4. Summary: "Shoot all the bluejays you want, if you can hit 'em, but remember it's a sin to kill a mockingbird," Atticus Finch told his children. "It's a crime to shoot a bluejay and not a bird," he said.
          Thank you, Inventic! for the Opportunity          


In [6]:
user_passage = input("Paste the book passage here:\n")

analyzer.execute_report(user_passage)

Paste the book passage here:
The location of the hotel was perfect, right in the heart of the city, and the lobby looked beautiful. However, the room itself was quite small and the air conditioning was loud, making it hard to sleep. The staff were friendly, but the breakfast options were limited and cold. It was a decent stay, but probably not worth the high price tag.

          Book Passage Analysis Report By Vikash Kumar          
1. Total Word Count:    62
2. Predominant Emotion: Neutral
3. Possible Books:      The Alchemist, Man's Search for Meaning, To Kill a Mockingbird

 Please wait some time, the model is analyzing for Comprehensive Summary 

4. Summary: The location of the hotel was perfect, right in the heart of the city. The staff were friendly, but the breakfast options were limited and cold. It was a decent stay, but probably not worth the high price tag.
          Thank you, Inventic! for the Opportunity          


In [7]:
user_passage = input("Paste the book passage here:\n")

analyzer.execute_report(user_passage)

Paste the book passage here:
"Extremely disappointed with this blender. After only two uses, the motor started smoking and smelled like burning plastic. It barely crushes ice, let alone makes a smooth smoothie. To make matters worse, the lid doesn't seal properly, resulting in a huge mess on my counter. I will be returning this product immediately and asking for a full refund."

          Book Passage Analysis Report By Vikash Kumar          
1. Total Word Count:    59
2. Predominant Emotion: Disgust
3. Possible Books:      The Alchemist, Man's Search for Meaning, To Kill a Mockingbird

 Please wait some time, the model is analyzing for Comprehensive Summary 

4. Summary: "Extremely disappointed with this blender. After only two uses, the motor started smoking and smelled like burning plastic" "It barely crushes ice, let alone makes a smooth smoothie" "I will be returning this product immediately and asking for a full refund"
          Thank you, Inventic! for the Opportunity          

In [8]:
user_passage = input("Paste the book passage here: \n")

analyzer.execute_report(user_passage)

Paste the book passage here: 
I am absolutely thrilled with the new wireless headphones! The sound quality is crisp, the bass is deep, and they are incredibly comfortable to wear for long periods. Battery life easily lasts throughout my workday. Shipping was fast, and the customer service team was very helpful with my questions. Highly recommended!"

          Book Passage Analysis Report By Vikash Kumar          
1. Total Word Count:    51
2. Predominant Emotion: Joy
3. Possible Books:      The Alchemist, Man's Search for Meaning, To Kill a Mockingbird

 Please wait some time, the model is analyzing for Comprehensive Summary 

4. Summary: The sound quality is crisp, the bass is deep, and they are incredibly comfortable to wear for long periods. Battery life easily lasts throughout my workday. Shipping was fast, and the customer service team was very helpful with my questions. Highly recommended!"
          Thank you, Inventic! for the Opportunity          


In [9]:
user_passage = input("Paste the book passage here: \n")

analyzer.execute_report(user_passage)

Paste the book passage here: 
While suffering is often an unavoidable part of the human experience, it does not have to be meaningless. By shifting our perspective and finding meaning within our trials, we can discover a lasting, inner freedom that no external circumstance can take away."

          Book Passage Analysis Report By Vikash Kumar          
1. Total Word Count:    42
2. Predominant Emotion: Sadness
3. Possible Books:      Man's Search for Meaning by Viktor Frankl, The Choice

 Please wait some time, the model is analyzing for Comprehensive Summary 

4. Summary: While suffering is often an unavoidable part of the human experience, it does not have to be meaningless. By shifting our perspective and finding meaning within our trials, we can discover a lasting, inner freedom that no external circumstance can take away.
          Thank you, Inventic! for the Opportunity          
